In [1]:
import re
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import one_hot
from sklearn.metrics import confusion_matrix,accuracy_score
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,LSTM,Dense,Dropout

nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


False

In [2]:
df = pd.read_csv('/Users/akhilajoshi/Python_Coding/fakenews/train.csv')
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.head(10)

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [3]:
X = df['title']
y = df['label']

In [4]:
ps = PorterStemmer()
corpus = []

for i in range(len(X)):
    text = X[i]
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(t) for t in text if t not in stopwords.words('english')]
    corpus.append(' '.join(text))

In [5]:
vocab_size = 5000
sent_len = 20
one_hot_encoded = [one_hot(x,vocab_size) for x in corpus]
one_hot_encoded = pad_sequences(one_hot_encoded,maxlen=sent_len)
one_hot_encoded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  642,
        515, 4343,  616, 4842, 1074, 2147,  974, 1674,  756], dtype=int32)

In [6]:
X = np.array(one_hot_encoded)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
no_of_output_features = 40

model = Sequential()
model.add(Embedding(vocab_size,no_of_output_features,input_length=sent_len))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=64,epochs=20)

Epoch 1/20
192/192 [==============================] - 4s 16ms/step - loss: 0.5293 - accuracy: 0.7606 - val_loss: 0.3075 - val_accuracy: 0.8756
Epoch 2/20
192/192 [==============================] - 3s 16ms/step - loss: 0.2557 - accuracy: 0.9083 - val_loss: 0.5203 - val_accuracy: 0.9079
Epoch 3/20
192/192 [==============================] - 3s 16ms/step - loss: 0.2069 - accuracy: 0.9380 - val_loss: 0.3808 - val_accuracy: 0.9178
Epoch 4/20
192/192 [==============================] - 3s 15ms/step - loss: 0.2224 - accuracy: 0.9264 - val_loss: 0.3776 - val_accuracy: 0.8973
Epoch 5/20
192/192 [==============================] - 3s 16ms/step - loss: 0.1925 - accuracy: 0.9553 - val_loss: 0.5016 - val_accuracy: 0.9176
Epoch 6/20
192/192 [==============================] - 3s 16ms/step - loss: 0.1624 - accuracy: 0.9536 - val_loss: 0.4980 - val_accuracy: 0.9173
Epoch 7/20
192/192 [==============================] - 3s 15ms/step - loss: 0.1444 - accuracy: 0.9660 - val_loss: 0.4846 - val_accuracy: 0.9167

In [9]:
pred = (model.predict(X_test) > 0.5).astype("int32")
confusion_matrix(y_test,pred)

189/189 [==============================] - 1s 3ms/step


array([[3118,  301],
       [ 231, 2385]])

In [10]:
accuracy_score(y_test,pred)

0.911847555923778